In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [2]:
df = pd.read_table(filepath_or_buffer='train.dat', header=None)
df1 = pd.read_table(filepath_or_buffer='test.dat', header=None)
vals = df.iloc[:,:].values
vals1 = df1.iloc[:,:].values
reviewstest=[]
for i in range(len(vals1)):
    reviewstest.append(vals1[i,:])
#reviewstest1 = reviewstest.tolist()
reviews=[]
sts=[]
#print vals
for i in range(len(vals)):
    if(i == len(vals)):
        break
    else:
        reviews.append(vals[i,1])
for i in range(len(vals)):
    if(i == len(vals)):
        break
    else:
        sts.append(vals[i,0])

In [3]:
rvs=[]
rvste=[]
#sentiments stms
#for i in range(len(reviews)):
 #rvs.append(reviews[i].reviews('<br /><br />'))
for i in range(len(reviews)):
    rvs.append(reviews[i].split('<br /><br />'))
#y = isinstance(rvs, list)
#print y
for i in range(len(reviewstest)):
    rvste.append(reviewstest[i].tolist())

In [4]:
def cmer(rv, c = 3):
    str1 = ''.join(map(str, rv))
    str1 = str1.lower()
    v = []
    n = len(str1)
    for i in range(0, n-c+1):
        v.append(str1[i:i+c])
    return v

In [5]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, review="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )
        
def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
def namesToMatrix(revs, c):
    #for rv in revs:
     #st = ''.join(map(str, rv))
      #docs = [cmer(st, c)]
    docs = [cmer(r,c) for r in revs]
    return build_matrix(docs)

In [6]:
# tc = 0
# k = 3
# pr=[]
# mat1 = namesToMatrix(rvs, 3)
# mat2 = namesToMatrix(rvste, 3)
# m1 = csr_l2normalize(mat1, copy=True)
# m2 = csr_l2normalize(mat2, copy=True)
# dots = (m1[0].T).dot(m2[0])
# sims = list(zip(dots.indices, dots.data))
# sims.sort(key=lambda x: x[1], reverse=True)
# tc = Counter(sts[s[0]] for s in sims[:k]).most_common(2)
# if len(tc) < 2 or tc[0][1] > tc[1][1]:
#             # majority vote
#     tc = tc[0][0]
#         # tie break
#     tc = defaultdict(float)
#     for s in sims[:k]:
#         tc[sts[s[0]]] += s[1]
#         pr.append(sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0])


In [7]:
#sts[518]

In [6]:
def classifyNames(rvs, sts, rvste, c=3, k=3):
    r""" Classify names using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """
    docs = [cmer(r, c) for r in rvs]
    docs1 = [cmer(r, c) for r in rvste]
    mat = build_matrix(docs)
    mat1 = build_matrix(docs1)
    #ststr = sts
    # since we're using cosine similarity, normalize the vectors
    m = csr_l2normalize(mat, copy=True)
    m1 = csr_l2normalize(mat1, copy=True)
    def classify(x, train, sts1):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = (x.T).dot(train)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+1' if np.random.rand() > 0.5 else '-1'
        sims.sort(key=lambda x: x[1], reverse=True)
        sts1 = sts
        tc = Counter(sts1[s[0]] for s in sims[:k] if s[0] < 25000 and s[0] > -1).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            if s[0] < 25000 and s[0] > -1:
                tc[sts1[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        
    macc = 0.0
    
        # split data into training and testing
        #train, ststr = splitData(mat, sts, f+1, d)
        # predict the class of each test sample
    sts1 = sts 
    stspr = [ classify(m1[i,:], m[i], sts1) for i in range(m1.shape[0]) ]
        
        # compute the accuracy of the prediction
    stspr1=[]
    for i in range(len(stspr)):
        stspr1.append(stspr[i])
        
    f = open("r0.dat", "w")
    f.write("\n".join(map(lambda x: str(x), stspr)))
    f.close()
    return stspr1

In [8]:
pr = classifyNames(rvs, sts, rvste, c = 3, k = 2)

In [11]:
len(pr)

25000